In [161]:
import pandas as pd
from bs4 import BeautifulSoup

In [163]:
dd = pd.read_pickle('/home/charubaiel/workflow/MLOpsini_project/tmp5cpvthqc/storage/cian_dataframe')

In [174]:
dd['title'].str.split(',',expand=True,n=3).head()

,0,1,2,3
0,2-комн. кв.,65,4 м²,15/24 этаж
1,2-комн. кв.,44,5 м²,2/2 этаж
2,2-комн. кв.,48 м²,4/23 этаж,None
3,2-комн. кв.,47,6 м²,3/16 этаж
4,3-комн. кв.,99 м²,19/22 этаж,None


In [183]:
dd['title'].str.replace(',(?=\d)','.').str.split(',',expand=True).head()

/tmp/ipykernel_21848/3191180561.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dd['title'].str.replace(',(?=\d)','.').str.split(',',expand=True).head()


,0,1,2
0,2-комн. кв.,65.4 м²,15/24 этаж
1,2-комн. кв.,44.5 м²,2/2 этаж
2,2-комн. кв.,48 м²,4/23 этаж
3,2-комн. кв.,47.6 м²,3/16 этаж
4,3-комн. кв.,99 м²,19/22 этаж


In [13]:
with open('cian.html') as b:
    html_ = BeautifulSoup(b.read())

In [ ]:
for item in html_data

In [ ]:
@asset(name = 'avito_dataframe',
       description='Выделение только новых объявлений',
       group_name='Extract')
def convert_html_2_df_cian(context,cian_page_list:list) -> pd.DataFrame:

    result_list = []
    for page in cian_page_list:
        
        items = BeautifulSoup(page, features='lxml').findAll('div',{'data-testid':'offer-card'})

        tmp_result = [get_cian_item_info(item=item) for item in items]
        result_list.extend(tmp_result)
        
    result = pd.DataFrame(result_list)

    return result


In [129]:

def get_cian_item_info(item:BeautifulSoup):
    item_desc = {}
    item_desc['datetime'] = pd.to_datetime('now',utc=True).timestamp()
    item_desc['title'] = item.find('span',{'data-mark':'OfferTitle'}).text
    item_desc['price'] = item.find('span',{'data-mark':'MainPrice'}).text.replace('\xa0₽','').replace(' ','')
    item_desc['publish_delta'] = item.find('div',{'class':'_93444fe79c--relative--IYgur'}).text.replace('\n','').replace('  ',' ')
    item_desc['url'] = item.find('div',{'data-name':'LinkArea'}).a['href']
    item_desc['id'] = item_desc['url'].split('/')[-2]
    item_desc['text'] = item.find('div',{'data-name':'Description'}).text
    item_desc.update(dict(zip(('Город','Округ','Метро','Район','Улица','Дом'),[step.text for step in item.findAll('a',{'data-name':'GeoLabel'})])))
    item_desc['metro_branch'] = item.find('div',{'data-name':'UndergroundIconWrapper'})['style'].replace('color: rgb','').replace(';','')
    item_desc['metro_name'] = [i for i in item.find('div',{'data-name':'SpecialGeo'}).text.split('\n') if i !=''][0]
    item_desc['metro_dist'] = [i for i in item.find('div',{'data-name':'SpecialGeo'}).text.split('\n') if i !=''][1]
    item_desc['img_list'] = [img['src'] for img in item.find('div',{'data-name':'Gallery'}).findAll('img') if img['src'].endswith('.jpg')]

    return item_desc

In [123]:
item_ = html_.findAll('div',{'data-testid':'offer-card'})[0]

In [148]:
from dagster import asset

In [157]:
@asset(name = 'featurized_cian_data',
       description='Дополнение данных простыми фичами',
       group_name='Featurize')
def featuring_cian_data(cian_dataframe:pd.DataFrame)->pd.DataFrame:

    cian_dataframe['price'] = cian_dataframe['price'].astype(float)
    cian_dataframe[['rooms','m2','floor']] = cian_dataframe['title'].str.split(',',expand=True)
    cian_dataframe['m2'] = cian_dataframe['m2'].str.extract('(\d+).*м²').astype(float)
    cian_dataframe[['floor','max_floor']] = cian_dataframe['floor'].str.extract('(\d+/\d+).*эт')[0].str.split('/',expand=True).astype(int)
    cian_dataframe['text'] = cian_dataframe['text'].str.replace('\n+','').str.replace(' +',' ')
    cian_dataframe['rubm2'] = cian_dataframe['price'] / cian_dataframe['m2']
    cian_dataframe.drop(['title'],axis=1,inplace=True)

    return cian_dataframe

In [158]:
dd = pd.DataFrame([get_cian_item_info(item=item_)])

In [159]:
new_dd = featuring_cian_data(dd)

/tmp/ipykernel_21848/2407911243.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cian_dataframe['text'] = cian_dataframe['text'].str.replace('\n+','').str.replace(' +',' ')


In [160]:
new_dd.T

,0
datetime,1680638353.991387
price,21500000.0
publish_delta,45 минут назад
url,https://www.cian.ru/sale/flat/285761548/
id,285761548
text,Предлагается к продаже прекрасная видовая дву...
Город,Москва
Округ,НАО (Новомосковский)
Метро,м. Говорово
Район,Московский поселение
